# COVID 19 Daily Cases Open Data

##### Author: Viridiana Perez
##### Date: July 28 2020
##### Last modified: July 28 2020
###### Source: "COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University" or "JHU CSSE COVID-19 Data"  https://github.com/CSSEGISandData/COVID-19

In [37]:
# Importing libraries
import requests as r
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
#do this if you don't have the latest pandas version
# from pandas.io.json import json_normalize

Dowload data with API

In [6]:
# Get API link with confimred cases 
API_Link="http://covid19api.herokuapp.com/confirmed"
# Pull data 
json_data = r.get(API_Link).json()

Now that the pandas have the dataframe, using locations column we will format the data 

In [8]:
confirmed_df = pd.json_normalize(json_data,record_path=['locations'])

Reset the index to the country column

In [9]:
confirmed_df.set_index('country', inplace=True)

This is what the data looks like

In [12]:
confirmed_df.head()

,country_code,latest,province,coordinates.latitude,coordinates.longitude,history.1/22/20,history.1/23/20,history.1/24/20,history.1/25/20,history.1/26/20,...,history.7/25/20,history.7/26/20,history.7/27/20,history.7/3/20,history.7/4/20,history.7/5/20,history.7/6/20,history.7/7/20,history.7/8/20,history.7/9/20
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,AF,36263,nan,33.93911,67.709953,0,0,0,0,0,...,36036,36157,36263,32324,32672,32951,33190,33384,33594,33908
Albania,AL,4880,nan,41.15330,20.168300,0,0,0,0,0,...,4637,4763,4880,2752,2819,2893,2964,3038,3106,3188
Algeria,DZ,27973,nan,28.03390,1.659600,0,0,0,0,0,...,26764,27357,27973,15070,15500,15941,16404,16879,17348,17808
Andorra,AD,907,nan,42.50630,1.521800,0,0,0,0,0,...,897,897,907,855,855,855,855,855,855,855
Angola,AO,950,nan,-11.20270,17.873900,0,0,0,0,0,...,916,932,950,328,346,346,346,386,386,396


In [11]:
json_data['locations'][0]['history']

{'1/22/20': 0,
 '1/23/20': 0,
 '1/24/20': 0,
 '1/25/20': 0,
 '1/26/20': 0,
 '1/27/20': 0,
 '1/28/20': 0,
 '1/29/20': 0,
 '1/30/20': 0,
 '1/31/20': 0,
 '2/1/20': 0,
 '2/10/20': 0,
 '2/11/20': 0,
 '2/12/20': 0,
 '2/13/20': 0,
 '2/14/20': 0,
 '2/15/20': 0,
 '2/16/20': 0,
 '2/17/20': 0,
 '2/18/20': 0,
 '2/19/20': 0,
 '2/2/20': 0,
 '2/20/20': 0,
 '2/21/20': 0,
 '2/22/20': 0,
 '2/23/20': 0,
 '2/24/20': 1,
 '2/25/20': 1,
 '2/26/20': 1,
 '2/27/20': 1,
 '2/28/20': 1,
 '2/29/20': 1,
 '2/3/20': 0,
 '2/4/20': 0,
 '2/5/20': 0,
 '2/6/20': 0,
 '2/7/20': 0,
 '2/8/20': 0,
 '2/9/20': 0,
 '3/1/20': 1,
 '3/10/20': 5,
 '3/11/20': 7,
 '3/12/20': 7,
 '3/13/20': 7,
 '3/14/20': 11,
 '3/15/20': 16,
 '3/16/20': 21,
 '3/17/20': 22,
 '3/18/20': 22,
 '3/19/20': 22,
 '3/2/20': 1,
 '3/20/20': 24,
 '3/21/20': 24,
 '3/22/20': 40,
 '3/23/20': 40,
 '3/24/20': 74,
 '3/25/20': 84,
 '3/26/20': 94,
 '3/27/20': 110,
 '3/28/20': 110,
 '3/29/20': 120,
 '3/3/20': 1,
 '3/30/20': 170,
 '3/31/20': 174,
 '3/4/20': 1,
 '3/5/20': 1,
 

We have to remove the coordinates and history prefix to make the plot and manipulate the data easily so we need to define a function to remove the prefix 

In [28]:
# Define a function to drop the history.prefix
# Create function drop_prefix
def drop_prefix(self, prefix):
    self.columns = self.columns.str.lstrip(prefix)
    return self

# Call function which we renamed to a shorter name so is easier to write it, like an alias
pd.core.frame.DataFrame.drop_prefix = drop_prefix

In [40]:
# Define function which removes history. prefix, and orders the column dates in ascending order
def order_dates(flat_df):

    # Drop prefix
    flat_df.drop_prefix('history.')
    flat_df.drop_prefix("coordinates.")
    # Isolate dates columns
    flat_df.iloc[:,6:].columns = pd.to_datetime(flat_df.iloc[:,6:].columns)
    # Transform to datetim format
    sub = flat_df.iloc[:,6:]
    sub.columns = pd.to_datetime(sub.columns)
    # Sort
    sub2 = sub.reindex(sorted(sub.columns), axis=1)
    sub3 = flat_df.reindex(sorted(flat_df.columns),axis=1).iloc[:,-5:]
    # Concatenate
    final = pd.concat([sub2,sub3], axis=1, sort=False)
    return final

In [47]:
# Apply function
final_confirmed = order_dates(confirmed_df)
final.head()

,2020-01-23 00:00:00,2020-01-24 00:00:00,2020-01-25 00:00:00,2020-01-26 00:00:00,2020-01-27 00:00:00,2020-01-28 00:00:00,2020-01-29 00:00:00,2020-01-30 00:00:00,2020-01-31 00:00:00,2020-02-01 00:00:00,...,2020-07-23 00:00:00,2020-07-24 00:00:00,2020-07-25 00:00:00,2020-07-26 00:00:00,2020-07-27 00:00:00,latest,latitude,longitude,province,untry_code
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,35928,35981,36036,36157,36263,36263,33.93911,67.709953,nan,AF
Albania,0,0,0,0,0,0,0,0,0,0,...,4466,4570,4637,4763,4880,4880,41.15330,20.168300,nan,AL
Algeria,0,0,0,0,0,0,0,0,0,0,...,25484,26159,26764,27357,27973,27973,28.03390,1.659600,nan,DZ
Andorra,0,0,0,0,0,0,0,0,0,0,...,889,897,897,897,907,907,42.50630,1.521800,nan,AD
Angola,0,0,0,0,0,0,0,0,0,0,...,851,880,916,932,950,950,-11.20270,17.873900,nan,AO


In [42]:
confirmed_df.iloc[:,6:]

,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,...,7/25/20,7/26/20,7/27/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,36036,36157,36263,32324,32672,32951,33190,33384,33594,33908
Albania,0,0,0,0,0,0,0,0,0,0,...,4637,4763,4880,2752,2819,2893,2964,3038,3106,3188
Algeria,0,0,0,0,0,0,0,0,0,0,...,26764,27357,27973,15070,15500,15941,16404,16879,17348,17808
Andorra,0,0,0,0,0,0,0,0,0,0,...,897,897,907,855,855,855,855,855,855,855
Angola,0,0,0,0,0,0,0,0,0,0,...,916,932,950,328,346,346,346,386,386,396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sao Tome and Principe,0,0,0,0,0,0,0,0,0,0,...,862,863,865,719,719,720,721,724,724,726
Yemen,0,0,0,0,0,0,0,0,0,0,...,1674,1681,1691,1240,1248,1265,1284,1297,1318,1356
Comoros,0,0,0,0,0,0,0,0,0,0,...,340,354,354,309,309,311,311,311,313,314


In [48]:
condition=final.index=='Canada'
final[condition]

,2020-01-23 00:00:00,2020-01-24 00:00:00,2020-01-25 00:00:00,2020-01-26 00:00:00,2020-01-27 00:00:00,2020-01-28 00:00:00,2020-01-29 00:00:00,2020-01-30 00:00:00,2020-01-31 00:00:00,2020-02-01 00:00:00,...,2020-07-23 00:00:00,2020-07-24 00:00:00,2020-07-25 00:00:00,2020-07-26 00:00:00,2020-07-27 00:00:00,latest,latitude,longitude,province,untry_code
country,,,,,,,,,,,,,,,,,,,,,
Canada,0,0,0,0,0,0,0,0,0,0,...,9975,10086,10086,10086,10390,10390,53.9333,-116.5765,Alberta,CA
Canada,0,0,0,0,0,1,1,1,1,1,...,3392,3392,3419,3419,3500,3500,53.7267,-127.6476,British Columbia,CA
Canada,0,0,0,0,0,0,0,0,0,0,...,13,13,13,13,13,13,0.0000,0.0000,Grand Princess,CA
Canada,0,0,0,0,0,0,0,0,0,0,...,375,384,388,394,400,400,53.7609,-98.8139,Manitoba,CA
Canada,0,0,0,0,0,0,0,0,0,0,...,170,170,170,170,170,170,46.5653,-66.4619,New Brunswick,CA
Canada,0,0,0,0,0,0,0,0,0,0,...,264,265,265,266,266,266,53.1355,-57.6604,Newfoundland and Labrador,CA
Canada,0,0,0,0,0,0,0,0,0,0,...,1067,1067,1067,1067,1067,1067,44.6820,-63.7443,Nova Scotia,CA
Canada,0,0,0,1,1,1,1,1,3,3,...,39936,40341,40457,40558,40673,40673,51.2538,-85.3232,Ontario,CA
Canada,0,0,0,0,0,0,0,0,0,0,...,36,36,36,36,36,36,46.5107,-63.4168,Prince Edward Island,CA


We're gonna modify the columns so we don't see gaps on the dates, we can sort them so they're in order

In [54]:
country = "Canada"
transposed_final = final[final.index=='Canada'].set_index("province").T.iloc[:-4,]
transposed_final.head()
by_prov["TotalDailyCase"] = by_prov.sum(axis=1)

In [55]:
transposed_final ["TotalDailyCase"] = transposed_final.sum(axis=1)

In [56]:
transposed_final.tail()

province,Alberta,British Columbia,Grand Princess,Manitoba,New Brunswick,Newfoundland and Labrador,Nova Scotia,Ontario,Prince Edward Island,Quebec,Saskatchewan,Diamond Princess,Northwest Territories,Yukon,TotalDailyCase
2020-07-23 00:00:00,9975,3392,13,375,170,264,1067,39936,36,58080,1072,0,5,13,114398.0
2020-07-24 00:00:00,10086,3392,13,384,170,265,1067,40341,36,58243,1099,0,5,14,115115.0
2020-07-25 00:00:00,10086,3419,13,388,170,265,1067,40457,36,58414,1136,0,5,14,115470.0
2020-07-26 00:00:00,10086,3419,13,394,170,266,1067,40558,36,58583,1178,0,5,14,115789.0
2020-07-27 00:00:00,10390,3500,13,400,170,266,1067,40673,36,58728,1209,0,5,14,116471.0


In [62]:
px.scatter(transposed_final,
          x=transposed_final.index,
          y='TotalDailyCase',
          title='Total (cumulative) COVID19 Reported Infections in Canada',
          labels={"x":"Time(daily)",
           "TotalDailyCase":"Total number of reported cases"})

To get granularity about the cases we can look at the non cumulative cases 


In [63]:
transposed_final.diff(axis=0)

province,Alberta,British Columbia,Grand Princess,Manitoba,New Brunswick,Newfoundland and Labrador,Nova Scotia,Ontario,Prince Edward Island,Quebec,Saskatchewan,Diamond Princess,Northwest Territories,Yukon,TotalDailyCase
2020-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2020-01-25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2020-01-26,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1.0
2020-01-27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-23,247,64,0,1,0,0,0,112,0,142,42,0,0,0,608.0
2020-07-24,111,0,0,9,0,1,0,405,0,163,27,0,0,1,717.0
2020-07-25,0,27,0,4,0,0,0,116,0,171,37,0,0,0,355.0
2020-07-26,0,0,0,6,0,1,0,101,0,169,42,0,0,0,319.0


In [64]:
non_cumulative =transposed_final.diff(axis=0)

In [68]:
px.line(non_cumulative,
       x=non_cumulative.index,
       y='TotalDailyCase',
       title="Daily (non-cumulative) COVID19 Cases in canada",
       labels={'x':'Timee (days)',
              "TotalDailyCases":'Non cumulative reported daily cases'})